In [5]:
import numpy as np 
import matplotlib.pyplot as plt
import csv 
import pickle
from scipy import stats
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, f1_score, recall_score
from sklearn.linear_model import LogisticRegression

## Extract train/test features from Image-Only model

In [3]:
# image_feats_train = 
# tube_labels_train = 
# pmx_labels_train = 
# probs_train = 
# time_all = 



# image_feats_test = 
# tube_labels_test = 
# pmx_labels_test = 
# probs_test = 

## train few-shot LR models with gaze features

In [10]:
k = 20
gaze_feature = time_all
label_mask = pmx_labels_train == 1
label_mask_test = pmx_labels_test > -1


thresh1 = np.sort(gaze_feature[label_mask])[-k]
thresh2 = np.sort(gaze_feature[label_mask])[k]
label_mask_1 = gaze_feature[label_mask] >= thresh1
label_mask_0 = gaze_feature[label_mask] < thresh2

X_train = np.concatenate((image_feats_train[label_mask][label_mask_1],image_feats_train[label_mask][label_mask_0]))
y_train = np.array([1]*label_mask_1.sum() + [0]*label_mask_0.sum())

print(X_train.shape)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
test_slices = clf.predict(image_feats_test[label_mask_test])
print(f"{test_slices.sum()}/{len(test_slices)}")


(40, 2048)
87/1000


## View performance gaps in test set

In [13]:
thresh = 0.3 #373 # this threshold achieves 0.55 recall for overall
preds_test = (probs_test > thresh)
overall_acc = (preds_test[label_mask_test] == pmx_labels_test[label_mask_test]).mean()
#print(f"Overall accuracy for class: {100*overall_acc:.2f}")
print(f"Overall AUROC: {100*roc_auc_score(pmx_labels_test, probs_test):.2f}")


mask = test_slices.astype(bool)
acc_0 = (preds_test[label_mask_test][mask] == pmx_labels_test[label_mask_test][mask]).mean()
#print(f"Accuracy for class on slice 0: {100*acc_0:.2f}")
print(f"Mask1 AUROC: {100*roc_auc_score(pmx_labels_test[mask], probs_test[mask]):.2f}")


mask = (1-test_slices).astype(bool)
acc_1 = (preds_test[label_mask_test][mask] == pmx_labels_test[label_mask_test][mask]).mean()
#print(f"Accuracy for class on slice 1: {100*acc_1:.2f}")
print(f"Mask2 AUROC: {100*roc_auc_score(pmx_labels_test[mask], probs_test[mask]):.2f}")


Overall AUROC: 90.86
Mask1 AUROC: 71.22
Mask2 AUROC: 88.42
